In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/data-storm-10/DATA STORM 1.0 - First Round Competition Guidlines.pdf
/kaggle/input/data-storm-10/credit_card_default_test.csv
/kaggle/input/data-storm-10/credit_card_default_train.csv
/kaggle/input/data-storm-10/Credit_card_default - Business Problem - Assessment Criteria - Data Dictionary.xlsx


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
credit_card_default_train=pd.read_csv('/kaggle/input/data-storm-10/credit_card_default_train.csv')
credit_card_default_test=pd.read_csv('/kaggle/input/data-storm-10/credit_card_default_test.csv')
ccdt = credit_card_default_test

# **Featuer Engineering**

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = credit_card_default_train.drop('NEXT_MONTH_DEFAULT', axis =1)
y = credit_card_default_train['NEXT_MONTH_DEFAULT']


X.drop('Client_ID', axis = 1, inplace = True)
test_client_ids =  credit_card_default_test['Client_ID']
credit_card_default_test.drop('Client_ID', axis = 1, inplace = True)

def balance_limit_encoder(value):
    last = value[-1]
    value = value[:-1]
    int_val = 0
    if last == 'K':
        int_val = float(value) * 1e3
    elif last == 'M':
        int_val = float(value) * 1e6
    return int_val
    
    
#Label Encoding

unused_cols = ['Balance_Limit_V1', 'EDUCATION_STATUS', 'MARITAL_STATUS', 'AGE']

marital_status_encoder = LabelEncoder()
education_status_encoder = LabelEncoder()
age_encoder = LabelEncoder()

# Fit Encoders
def fit_encoders(fit_data):
    marital_status_encoder.fit(fit_data['MARITAL_STATUS'])
    education_status_encoder.fit(fit_data['EDUCATION_STATUS'])
    age_encoder.fit(fit_data['AGE'])
    
# Transform Categorical Data
def preprocess(data):
    dataset = data.copy()
    dataset['Gender'] = pd.Series(data['Gender'].map(lambda df : 1 if df == 'M' else 0))
    dataset = dataset.assign(MARITAL_STATUS_ENC = marital_status_encoder.transform(data['MARITAL_STATUS']))
    dataset = dataset.assign(EDUCATION_STATUS_ENC = education_status_encoder.transform(data['EDUCATION_STATUS']))
    dataset = dataset.assign(AGE_ENC = age_encoder.transform(data['AGE']))
    dataset = dataset.assign(Balance_Limit_Enc = data['Balance_Limit_V1'].map(balance_limit_encoder))
    dataset.drop(unused_cols, axis= 1, inplace = True)
    return dataset

In [5]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,precision_score,recall_score
from xgboost import XGBClassifier
from sklearn.preprocessing import scale

fit_encoders(X)
X = preprocess(X)

X.head()

,Gender,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,...,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,MARITAL_STATUS_ENC,EDUCATION_STATUS_ENC,AGE_ENC,Balance_Limit_Enc
0,1,-1,-1,-1,-1,-1,-1,3248,3389,6004,...,3437,6004,39418,162772,0,538165,0,0,0,1000000.0
1,0,0,-1,-1,-1,-1,0,353351,151818,26948,...,151818,46200,43530,80811,942,33666,0,1,2,1000000.0
2,0,4,3,2,2,-2,-2,16681,16082,15477,...,0,0,0,0,0,0,1,1,0,100000.0
3,0,2,0,0,0,0,0,90457,92848,95193,...,3855,3890,3696,4620,4049,3918,1,0,0,200000.0
4,0,2,2,0,0,0,0,429556,419466,429785,...,0,20790,16170,17325,16401,17325,0,0,0,1000000.0


# **Training the model and predictions**

Due to imbalance nature of the dataset, upsamplig can be done using SMOTE

In [6]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE().fit_resample(X, y)

Using TensorFlow backend.


In [7]:
# test-train split

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size = 0.2,random_state=42)

In [8]:
# use XGBClassifier

from xgboost import XGBClassifier
clf=XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.16,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=400, 
                      reg_alpha = 0.3,
                      max_depth=12, 
                      gamma=10)
clf.fit(x_train,y_train)

In [16]:
#predictions

y_pred=clf.predict(x_val)
y_train_pred = clf.predict(x_train)

In [17]:
#evalustion 

from sklearn.metrics import classification_report

target_names = ['0','1']
print(classification_report(y_val, y_pred, target_names=target_names))


              precision    recall  f1-score   support

           0       0.81      0.84      0.83      3820
           1       0.83      0.80      0.81      3648

    accuracy                           0.82      7468
   macro avg       0.82      0.82      0.82      7468
weighted avg       0.82      0.82      0.82      7468



f1 score of 0 and 1 are quite similar beacuse of the upsampling done.

In [18]:
from sklearn.metrics import confusion_matrix, f1_score   

print('validation f1_score: ',f1_score(y_val,y_pred))
print('train f1_score: ',f1_score(y_train,y_train_pred))

validation f1_score:  0.813758389261745
train f1_score:  0.8660881402075888


Validation f1-score < train f1-score. This may be due to overfitting.

In [11]:
X_test = preprocess(credit_card_default_test)
X_test.head()

,Gender,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,...,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,MARITAL_STATUS_ENC,EDUCATION_STATUS_ENC,AGE_ENC,Balance_Limit_Enc
0,1,0,0,0,0,0,0,194188,198510,128316,...,13710,12816,195701,53246,17256,12991,1,2,0,500000.0
1,0,0,0,0,0,0,0,87115,89427,91823,...,3756,3842,3890,4620,4620,4620,1,2,0,200000.0
2,1,-1,-1,2,0,0,-1,2693,9573,8984,...,6930,0,2310,0,2310,40060,0,2,2,400000.0
3,1,1,-1,-1,-2,-2,-2,0,5775,5775,...,5775,5775,0,0,0,0,1,0,0,1500000.0
4,0,0,0,-2,-2,-2,-2,85257,0,0,...,0,0,0,0,0,0,0,1,2,100000.0


In [12]:
y_pred = clf.predict(X_test)

In [13]:
df = pd.DataFrame({
    'Client_ID' : test_client_ids,
    'NEXT_MONTH_DEFAULT': y_pred
})

In [14]:
df.to_csv('predictions.csv', index = False)